In [1]:
# Sbi
import sbi
from sbi.utils.get_nn_models import likelihood_nn
from sbi.analysis import pairplot
from sbi import inference
from sbi.vi.divergence_optimizers import ElboOptimizer
from sbi.vi.mixture_of_flows import MixtureFullGaussians

# Sbi benchmark
import sbibm
from sbibm.metrics import c2st

# torch and so
import torch
from torch.distributions.transforms import ComposeTransform
from torch import nn
import numpy as np

import matplotlib.pyplot as plt


D:\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# Loads a model...
def get_model(task_name):
    task = sbibm.get_task(task_name) 
    prior = task.prior_dist
    simulator = task.get_simulator()
    observation = task.get_observation(num_observation=1) 

    thetas = prior.sample((10000,))
    xs = simulator(thetas)


    inf = inference.SNLVI(prior)
    density_estimator = inf.append_simulations(thetas, xs).train(max_num_epochs=0)
    state_dict = torch.load(f"scripts/likelihood_models/{task_name}.net")
    density_estimator = inf._neural_net
    density_estimator.load_state_dict(state_dict())

    inf2 = inference.SNL(prior)
    density_estimator = inf2.append_simulations(thetas, xs).train(max_num_epochs=0)
    state_dict = torch.load(f"scripts/likelihood_models/{task_name}.net")
    density_estimator = inf2._neural_net
    density_estimator.load_state_dict(state_dict())
    return inf, inf2, task

In [5]:
inf_vi, inf_mcmc, task = get_model("two_moons")

In [8]:
observation = task.get_observation(1)

In [34]:
num_samples = list(range(1000, 11000, 1000))

In [36]:
import time

In [37]:
start = time.time()
posterior_mcmc = inf_mcmc.build_posterior()
posterior_mcmc.sample((10,), observation)
end = time.time()
print(end - start)



  0%|          | 0/50 [00:00<?, ?it/s]

Tuning bracket width...:   0%|          | 0/50 [00:00<?, ?it/s]

Tuning bracket width...:  20%|██        | 10/50 [00:00<00:01, 23.81it/s]

Tuning bracket width...:  40%|████      | 20/50 [00:00<00:01, 25.91it/s]

Tuning bracket width...:  60%|██████    | 30/50 [00:01<00:00, 29.35it/s]

Tuning bracket width...:  80%|████████  | 40/50 [00:01<00:00, 32.13it/s]

Tuning bracket width...: 100%|██████████| 50/50 [00:01<00:00, 30.36it/s]


  0%|          | 0/20 [00:00<?, ?it/s]

Generating samples:   0%|          | 0/20 [00:00<?, ?it/s]

Generating samples:  50%|█████     | 10/20 [00:03<00:03,  3.27it/s]

Generating samples: 100%|██████████| 20/20 [00:06<00:00,  3.17it/s]


  0%|          | 0/10 [00:00<?, ?it/s]

Generating samples:   0%|          | 0/10 [00:00<?, ?it/s]

Generating samples: 100%|██████████| 10/10 [00:03<00:00,  3.27it/s]11.05199646949768



In [35]:
num_samples

[1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

In [4]:
posterior_vi = inf_vi.build_posterior()
posterior_mcmc.sample((10000,), observation)

In [24]:
posterior_mcmc = inf_mcmc.build_posterior(mcmc_method = "slice_np_vectorized", mcmc_parameters = {
        "num_chains": 10,
        "thin": 10,
        "warmup_steps": 100,
        "init_strategy": "sir",
        "sir_batch_size": 1000,
        "sir_num_batches": 100,
    })

100%|██████████| 10100/10100 [01:20<00:00, 125.45it/s]


In [26]:
posterior_mcmc.sample((10000,), observation)

  0%|          | 0/110000 [00:00<?, ?it/s]Generating MCMC samples

  2%|▏         | 2220/110000 [00:14<12:28, 144.04it/s]

KeyboardInterrupt: 